In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import gurobipy as gp 
import numpy as np 
import networkx as nx
from opt_w_predictions import *
from sklearn.ensemble import RandomForestRegressor
from joblib import Parallel, delayed
from sklearn.linear_model import LinearRegression
from scipy.sparse import coo_matrix,csr_matrix


## Define optimization oracles for shortest path

In [2]:

def shortest_path_oracle(sources, destinations, start_node, end_node, quiet=True): 
    n_nodes = length(nodes)
    n_edges = length(sources)
    d_feasibleregion = length(sources)
    # Hard code sparse node-edge incidence matrix!
    I_vec = np.hstack([sources,destinations])
    J_vec = np.hstack([range(n_edges),range(n_edges)])
    V_vec = np.hstack([-np.ones(n_edges), np.ones(n_edges)])

    A_mat = coo_matrix((V_vec, (I_vec, J_vec)))
    A_mat = csr_matrix(A_mat)
    bvec = np.zeros(n_nodes)
    bvec[start_node] = -1; bvec[end_node] = 1; 
    m = gp.Model()
    if quiet: m.setParam("OutputFlag", 0)
    w = m.addMVar(n_edges, lb= 0, ub = 1)
    m.addConstrs(A_mat[i,:] @ w == bvec[i] for i in range(n_nodes) )
    def local_sp_oracle_jump(c):
        m.setObjective( c @ w, gp.GRB.MINIMIZE)
        m.optimize()
        z_ast = m.objVal
        w_ast = np.asarray([w[i].X for i in range(len(c))]).flatten()
        return [z_ast, w_ast]
    return local_sp_oracle_jump


def serializable_shortest_path_oracle(sources, destinations, start_node, end_node, quiet=True): 
    ''' Precomputing the model doesn't precompute for gurobi 
    '''
    n_nodes = length(nodes)
    n_edges = length(sources)
    d_feasibleregion = length(sources)
    # Hard code sparse node-edge incidence matrix!
    I_vec = np.hstack([sources,destinations])
    J_vec = np.hstack([range(n_edges),range(n_edges)])
    V_vec = np.hstack([-np.ones(n_edges), np.ones(n_edges)])

    A_mat = coo_matrix((V_vec, (I_vec, J_vec)))
    A_mat = csr_matrix(A_mat)
    bvec = np.zeros(n_nodes)
    bvec[start_node] = -1; bvec[end_node] = 1; 
    
    def serializable_local_sp_oracle_jump(c):
        m = gp.Model()
        if quiet: m.setParam("OutputFlag", 0)
        w = m.addMVar(n_edges, lb= 0, ub = 1)
        m.addConstrs(A_mat[i,:] @ w == bvec[i] for i in range(n_nodes) )
        m.setObjective( c @ w, gp.GRB.MINIMIZE)
        m.optimize()
        z_ast = m.objVal
        w_ast = np.asarray([w[i].X for i in range(len(c))]).flatten()
        return [z_ast, w_ast]
    return serializable_local_sp_oracle_jump

From the SPO paper: 
In the following set of experiments on the shortest path problem we described, we fix the number of features at p = 5 throughout and, as previously mentioned, use a 5 × 5 grid network, which implies that d = 40. Hence, in total there are pd = 200 parameters to estimate.

In [3]:
gurobiEnvOracle = gp.Env()
[sources, destinations, scalar_nodes, tuple_nodes] = convert_grid_to_list(5, 5)
grid_dim = 5
# start_node = 0; end_node = grid_dim**2-1
nodes = np.unique(list(set(sources).union(set(destinations))))
d_feasibleregion = length(sources)
start_node = scalar_nodes[(0,0)]
end_node = scalar_nodes[(4,4)]
sp_oracle = shortest_path_oracle(sources, destinations, start_node, end_node)
para_sp_oracle = serializable_shortest_path_oracle(sources, destinations, start_node, end_node)


Using license file /Users/connorlawless/gurobi.lic
Academic license - for non-commercial use only
{(0, 0): 0, (0, 1): 1, (0, 2): 2, (0, 3): 3, (0, 4): 4, (1, 0): 5, (1, 1): 6, (1, 2): 7, (1, 3): 8, (1, 4): 9, (2, 0): 10, (2, 1): 11, (2, 2): 12, (2, 3): 13, (2, 4): 14, (3, 0): 15, (3, 1): 16, (3, 2): 17, (3, 3): 18, (3, 4): 19, (4, 0): 20, (4, 1): 21, (4, 2): 22, (4, 3): 23, (4, 4): 24}
Using license file /Users/connorlawless/gurobi.lic
Academic license - for non-commercial use only


In [4]:

    # Generate Data
p_features=5
B_true = rand(1,0.5,size=(d_feasibleregion, p_features))
n_train = 500
n_test = 10000; n_holdout = 1000 
polykernel_degree_vec = [1, 2, 4, 6, 8]
polykernel_noise_half_width_vec = [0, 0.5]
polykernel_degree = polykernel_degree_vec[-1]
polykernel_noise_half_width = polykernel_noise_half_width_vec[1]

(X_train, c_train) = generate_poly_kernel_data_simple(B_true, n_train, polykernel_degree, polykernel_noise_half_width)
(X_validation, c_validation) = generate_poly_kernel_data_simple(B_true, n_holdout, polykernel_degree, polykernel_noise_half_width)
(X_test, c_test) = generate_poly_kernel_data_simple(B_true, n_test, polykernel_degree, polykernel_noise_half_width)

# Add intercept in the first row of X
# X is p x N 
X_train = vcat(ones(1,n_train), X_train)
X_validation = vcat(ones(1,n_holdout), X_validation)
X_test = vcat(ones(1,n_test), X_test)

# Get dimensions of input



In [5]:
def feasible_least_squares(regressor, x_star_regr, mu, c_train, X_train, sp_oracle, random_regr=False):
    '''
    One-step plug in 
    regressor: type of Scikit-Learn regressor
    x_star_regr: x^* - \hat x^* 
    c_train: training data, cost vector realizations
    X_train: training data 
    '''
    # Mix decision weights with uniform 
    weights = (mu*x_star_regr + (1-mu))*np.ones(c_train.shape)
    weighted_predictors = get_weighted_predictors(regressor, c_train, X_train, weights, random_regr=random_regr)
    [reweighted_regrets, reweighted_x_star_regr] = get_regret(weighted_predictors,
            X_train, c_train, X_test, c_test,sp_oracle, quiet=False)
    return [weighted_predictors, reweighted_regrets, reweighted_x_star_regr]
    
def run_replication_over_weights(data_params, X_test, c_test, mixture_weights, 
                                 regressor, sp_oracle, random_regr=False): 
    '''
    Run a replication under fixed data parameters 
    Assume pre-generated test dataset (to reduce noise in test evaluation)
    '''
    
    [n_train, n_test, n_holdout, polykernel_degree,polykernel_noise_half_width,B_true] = data_params
    [X_train, c_train,X_validation, c_validation] = generate_data(n_train, 
        n_test, n_holdout, polykernel_degree,polykernel_noise_half_width,B_true,gen_test=False)
    
    # learn initial predictor 
    predictors = get_weighted_predictors(regressor, c_train, X_train, random_regr=random_regr )
    [regrets, x_star_regr] = get_regret(predictors, X_train, c_train, X_test, c_test, sp_oracle)
    print(np.mean(np.sqrt(np.square(regrets))))
    print(x_star_regr.mean(axis=1))
    n_wghts = len(mixture_weights)

    original_regrets = np.mean(np.sqrt(np.square(regrets))); 
    reweighted_mean_regrets = np.zeros(n_wghts)
    for k in range(n_wghts): 
        [weighted_predictors, reweighted_regrets, reweighted_x_star_regr] = feasible_least_squares(
            regressor, x_star_regr, mixture_weights[k], c_train, X_train,sp_oracle,random_regr=random_regr)
#         print('reweighting ', mixture_weights[k])
        reweighted_mean_regrets[k] = np.mean(np.sqrt(np.square(reweighted_regrets)))
    return [original_regrets, reweighted_mean_regrets]



In [15]:
    [X_train, c_train,X_validation, c_validation] = generate_data(n_train, 
        n_test, n_holdout, polykernel_degree,polykernel_noise_half_width,B_true,gen_test=False)


In [17]:
sgd_tr_instance = []

for i in range(len(c_train.shape))

c_train.shape

(40, 500)

## Train predictors 

In [ ]:
run_replication_over_weights(data_params, X_test, c_test, mixture_weights, 
                                 regressor, sp_oracle, random_regr=False)

(6, 10000)

In [9]:
c_train.shape

(40, 500)

In [10]:
[X_train, c_train,X_validation, c_validation, X_train, X_test] = generate_data(n_train, 
            n_test, n_holdout, polykernel_degree,polykernel_noise_half_width,B_true,gen_test=True)

In [14]:
X_test.shape, c_test.shape

((40, 10000), (40, 10000))

In [6]:
import pickle 
#N_s = [ 100, 250, 500, 1000, 1500, 2000]
N_s = [ 100]

n_wghts = 5; mixture_weights = np.linspace(1.0/n_wghts,0.99,n_wghts)
n_reps = 12

# initialize choice of regressor
regressor = LinearRegression
random_regr=False
n_misp=len(polykernel_degree_vec)
original_regrets_ = np.zeros([n_misp,len(N_s), n_reps]); reweighted_regrets_ = np.zeros([n_misp,len(N_s), n_reps, n_wghts])


for misspec_ind, degree in enumerate(polykernel_degree_vec): 
    polykernel_degree = degree 
    for ind_n, en_train in enumerate(N_s): 
        print('n ,', en_train)
        data_params = [en_train, n_test, n_holdout, polykernel_degree,polykernel_noise_half_width,B_true]
        [X_train, c_train,X_validation, c_validation, X_train, X_test] = generate_data(n_train, 
            n_test, n_holdout, polykernel_degree,polykernel_noise_half_width,B_true,gen_test=True)

        # Each replication returns [original_regrets, reweighted_mean_regrets (n_wghts) ]
        res = Parallel(n_jobs=-1, verbose=20)(delayed(run_replication_over_weights)(
            data_params, X_test, c_test, mixture_weights, regressor, para_sp_oracle, random_regr=random_regr) for k in range(n_reps))

    #     [original_regrets, reweighted_regrets] = run_replication_over_weights(data_params, mixture_weights, regressor, sp_oracle, random_regr=random_regr)
        original_regrets_[misspec_ind,ind_n,:] = np.asarray([res[i][0] for i in range(n_reps)])
        reweighted_regrets_[misspec_ind,ind_n,:,:] = np.asarray([res[i][1] for i in range(n_reps)])

        exp_name = 'linear_regression_misspec_'+str(misspec_ind)
        res_ = dict({'original_regrets_': original_regrets_, 'reweighted_regrets_':reweighted_regrets_, 'data_params':data_params, 'N_s':N_s, 'mixture_weights':mixture_weights})
        pickle.dump(res_, open(exp_name+'-res_may18.p', 'wb'))


#     print('original regret, ', original_regrets)
#     print('reweighted regrets, ', reweighted_regrets)

n , 100


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   34.6s
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:   34.7s remaining:  2.9min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:   34.7s remaining:  1.7min
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:   34.8s remaining:  1.2min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   34.9s remaining:   48.9s
[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed:   35.0s remaining:   35.0s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   35.0s remaining:   25.0s
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed:   35.2s remaining:   17.6s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   54.0s remaining:   18.0s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:   54.1s remaining:   10.8s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   54.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed

n , 100


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:   32.5s remaining:  2.7min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:   32.6s remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:   32.6s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   32.7s remaining:   45.7s
[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed:   32.7s remaining:   32.7s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   32.8s remaining:   23.4s
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed:   32.9s remaining:   16.5s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   50.8s remaining:   16.9s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:   50.8s remaining:   10.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   50.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed

n , 100


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:   33.7s remaining:  2.8min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:   33.7s remaining:  1.7min
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:   33.7s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   33.7s remaining:   47.2s
[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed:   33.8s remaining:   33.8s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   33.8s remaining:   24.2s
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed:   34.0s remaining:   17.0s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   51.2s remaining:   17.1s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:   51.3s remaining:   10.3s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   51.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed

n , 100


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:   32.7s remaining:  2.7min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:   32.7s remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:   32.8s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   32.8s remaining:   45.9s
[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed:   32.9s remaining:   32.9s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   33.0s remaining:   23.6s
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed:   33.1s remaining:   16.5s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   50.5s remaining:   16.8s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:   50.7s remaining:   10.1s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   50.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed

n , 100


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   31.3s
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:   31.3s remaining:  2.6min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:   31.5s remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:   31.5s remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   31.5s remaining:   44.2s
[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed:   31.6s remaining:   31.6s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   31.6s remaining:   22.6s
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed:   31.8s remaining:   15.9s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   47.5s remaining:   15.8s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:   47.7s remaining:    9.5s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   47.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed

In [90]:
exp_name = 'linear_regression_'
res_ = dict({'original_regrets_': original_regrets_, 'reweighted_regrets_':reweighted_regrets_, 'data_params':data_params, 'N_s':N_s, 'mixture_weights':mixture_weights})
pickle.dump(res_, open(exp_name+'-res.p', 'wb'))

In [48]:
n_train = 250 
regressor = RandomForestRegressor
# [n_train, n_test, n_holdout, polykernel_degree,polykernel_noise_half_width,B_true] = data_params
[X_train, c_train,X_validation, c_validation, X_test, c_test] = generate_data(n_train, 
    n_test, n_holdout, polykernel_degree,polykernel_noise_half_width,B_true,gen_test=True)
predictors = get_weighted_predictors(regressor, c_train, X_train)
[regrets, x_star_regr] = get_regret(predictors, X_train, c_train, X_test, c_test, sp_oracle)
print(np.mean(np.sqrt(np.square(regrets))))
n_wghts = len(mixture_weights)

for k in range(n_wghts): 
    [weighted_predictors, reweighted_regrets, reweighted_x_star_regr] = feasible_least_squares(regressor, x_star_regr, mixture_weights[k], c_train, X_train,sp_oracle)
    print('reweighting ', mixture_weights[k])
    
    print(np.mean(np.sqrt(np.square(reweighted_regrets))))

0
0
reweighting  0.2
4743.91202177076
4854.331112751639
0
reweighting  0.4
4743.91202177076
4897.321988720906
0
reweighting  0.6000000000000001
4743.91202177076
5085.711101314495
0
reweighting  0.8
4743.91202177076
5334.150144065382
0
reweighting  1.0
4743.91202177076
6843.877078325886


In [22]:

    
# Feasible predict-then-optimize 
n_wghts = 5; mixture_weights = np.linspace(0,1,n_wghts)
for k in range(n_wghts): 
    weights = mixture_weights[k]*np.abs(x_star_regr) + (1- mixture_weights[k])*np.ones(c_train.shape)
    weighted_predictors = get_weighted_predictors(RandomForestRegressor, c_train, X_train,weights)
    [reweighted_regrets, reweighted_x_star_regr] = get_regret(weighted_predictors,X_train, c_train, X_test, c_test,quiet=False)

    print('reweighting ', mixture_weights[k])
    print(np.mean(np.sqrt(np.square(regrets))))
    print(np.mean(np.sqrt(np.square(reweighted_regrets))))




0
reweighting  0.0
3738.462547296788
3856.9007494036164
0
reweighting  0.25
3738.462547296788
3288.022802404781
0
reweighting  0.5
3738.462547296788
3635.9303441225484
0
reweighting  0.75
3738.462547296788
3770.76793113731
0
reweighting  1.0
3738.462547296788
2358.8529892644383


In [15]:
print(np.mean(np.sqrt(np.square(regrets))))
print(np.mean(np.sqrt(np.square(reweighted_regrets))))



0.7041773923402886
4.70906757162312


In [ ]:
x_star_regr.mean(axis=1)

In [ ]:
regrets

In [ ]:
## Get decision risks on previous dataset 

In [ ]:
"""
Run one instance of the shortest path experiment. Uses the reformulation approach.
`num_lambda` is the number of lambdas used on the grid for each method with regularization.
Returns a replication_results struct.
"""
function shortest_path_replication(grid_dim,
    n_train, n_holdout, n_test,
    p_features, polykernel_degree, polykernel_noise_half_width;
    num_lambda = 10, lambda_max = missing, lambda_min_ratio = 0.0001, regularization = :ridge,
    , gurobiEnvReform = Gurobi.Env(), different_validation_losses = false,
    include_rf = true)

    # Get oracle
    gurobiEnvOracle = Gurobi.Env()
    sources, destinations = convert_grid_to_list(grid_dim, grid_dim)
    d_feasibleregion = length(sources)
    sp_oracle = sp_flow_jump_setup(sources, destinations, 1, grid_dim^2; gurobiEnv = gurobiEnvOracle)
    sp_graph = shortest_path_graph(sources = sources, destinations = destinations,
        start_node = 1, end_node = grid_dim^2, acyclic = true)


    # Generate Data
    B_true = rand(Bernoulli(0.5), d_feasibleregion, p_features)

    (X_train, c_train) = generate_poly_kernel_data_simple(B_true, n_train, polykernel_degree, polykernel_noise_half_width)
    (X_validation, c_validation) = generate_poly_kernel_data_simple(B_true, n_holdout, polykernel_degree, polykernel_noise_half_width)
    (X_test, c_test) = generate_poly_kernel_data_simple(B_true, n_test, polykernel_degree, polykernel_noise_half_width)
    
    # Add intercept in the first row of X
    X_train = vcat(ones(1,n_train), X_train)
    X_validation = vcat(ones(1,n_holdout), X_validation)
    X_test = vcat(ones(1,n_test), X_test)

    # Get Hamming labels
    (z_train, w_train) = oracle_dataset(c_train, sp_oracle)
    (z_validation, w_validation) = oracle_dataset(c_validation, sp_oracle)
    (z_test, w_test) = oracle_dataset(c_test, sp_oracle)

    c_ham_train = ones(d_feasibleregion, n_train) - w_train
    c_ham_validation = ones(d_feasibleregion, n_holdout) - w_validation
    c_ham_test = ones(d_feasibleregion, n_test) - w_test

    # Put train + validation together
    X_both = hcat(X_train, X_validation)
    c_both = hcat(c_train, c_validation)
    c_ham_both = hcat(c_ham_train, c_ham_validation)
    train_ind = collect(1:n_train)
    validation_ind = collect((n_train + 1):(n_train + n_holdout))

    # Set validation losses
    if different_validation_losses
        spo_plus_val_loss = :spo_loss
        ls_val_loss = :least_squares_loss
        ssvm_val_loss = :hamming_loss
        absolute_val_loss = :absolute_loss
        huber_val_loss = :huber_loss
    else
        spo_plus_val_loss = :spo_loss
        ls_val_loss = :spo_loss
        ssvm_val_loss = :spo_loss
        absolute_val_loss = :spo_loss
        huber_val_loss = :spo_loss
    end

    ### Algorithms ###

    # SPO+
    best_B_SPOplus, best_lambda_SPOplus = validation_set_alg(X_both, c_both, sp_oracle; sp_graph = sp_graph,
        train_ind = train_ind, validation_ind = validation_ind,
        val_alg_parms = val_parms(algorithm_type = :sp_spo_plus_reform, validation_loss = spo_plus_val_loss),
        path_alg_parms = reformulation_path_parms(num_lambda = num_lambda, lambda_max = lambda_max, regularization = regularization,
            gurobiEnv = gurobiEnvReform, lambda_min_ratio = lambda_min_ratio, algorithm_type = :SPO_plus))

    # Least squares
    best_B_leastSquares, best_lambda_leastSquares = validation_set_alg(X_both, c_both, sp_oracle; sp_graph = sp_graph,
        train_ind = train_ind, validation_ind = validation_ind,
        val_alg_parms = val_parms(algorithm_type = :ls_jump, validation_loss = ls_val_loss),
        path_alg_parms = reformulation_path_parms(num_lambda = num_lambda, lambda_max = lambda_max, regularization = regularization,
            gurobiEnv = gurobiEnvReform, lambda_min_ratio = lambda_min_ratio, algorithm_type = :leastSquares))

    # SSVM Hamming
    best_B_SSVM, best_lambda_SSVM = validation_set_alg(X_both, c_ham_both, sp_oracle; sp_graph = sp_graph,
        train_ind = train_ind, validation_ind = validation_ind,
        val_alg_parms = val_parms(algorithm_type = :sp_spo_plus_reform, validation_loss = ssvm_val_loss),
        path_alg_parms = reformulation_path_parms(num_lambda = num_lambda, lambda_max = lambda_max, regularization = regularization,
            gurobiEnv = gurobiEnvReform, lambda_min_ratio = lambda_min_ratio, algorithm_type = :SSVM_hamming))

    # RF
    if include_rf
        rf_mods = train_random_forests_po(X_train, c_train;
            rf_alg_parms = rf_parms())
    end

    # Absolute
    best_B_Absolute, best_lambda_Absolute = validation_set_alg(X_both, c_both, sp_oracle; sp_graph = sp_graph,
        train_ind = train_ind, validation_ind = validation_ind,
        val_alg_parms = val_parms(algorithm_type = :ls_jump, validation_loss = absolute_val_loss),
        path_alg_parms = reformulation_path_parms(num_lambda = num_lambda, lambda_max = lambda_max, regularization = regularization,
            po_loss_function = :absolute, gurobiEnv = gurobiEnvReform, lambda_min_ratio = lambda_min_ratio, algorithm_type = :Absolute))

    # Huber
    best_B_fake, best_lambda_fake = validation_set_alg(X_both, c_both, sp_oracle; sp_graph = sp_graph,
        train_ind = train_ind, validation_ind = validation_ind,
        val_alg_parms = val_parms(algorithm_type = :ls_jump, validation_loss = ls_val_loss),
        path_alg_parms = reformulation_path_parms(lambda_max = 0.0001, regularization = regularization,
            num_lambda = 2, gurobiEnv = gurobiEnvReform, algorithm_type = :Huber_LS_fake))

    fake_ls_list = abs.(vec(c_train - best_B_fake*X_train))
    delta_from_fake_ls = median(fake_ls_list)

    best_B_Huber, best_lambda_Huber = validation_set_alg(X_both, c_both, sp_oracle; sp_graph = sp_graph,
        train_ind = train_ind, validation_ind = validation_ind,
        val_alg_parms = val_parms(algorithm_type = :ls_jump, validation_loss = huber_val_loss),
        path_alg_parms = reformulation_path_parms(num_lambda = num_lambda, lambda_max = lambda_max, regularization = regularization, po_loss_function = :huber,
            huber_delta = delta_from_fake_ls, lambda_min_ratio = lambda_min_ratio, gurobiEnv = gurobiEnvReform, algorithm_type = :Huber))


    # Baseline
    c_bar_train = mean(c_train, dims=2)


    ### Populate final results ###
    final_results = replication_results()

    final_results.SPOplus_spoloss_test = spo_loss(best_B_SPOplus, X_test, c_test, sp_oracle)
    final_results.LS_spoloss_test = spo_loss(best_B_leastSquares, X_test, c_test, sp_oracle)
    final_results.SSVM_spoloss_test = spo_loss(best_B_SSVM, X_test, c_test, sp_oracle)

    if include_rf
        rf_preds_test = predict_random_forests_po(rf_mods, X_test)
        final_results.RF_spoloss_test = spo_loss(Matrix(1.0I, d_feasibleregion, d_feasibleregion), rf_preds_test, c_test, sp_oracle)
    else
        final_results.RF_spoloss_test = missing
    end

    final_results.Absolute_spoloss_test = spo_loss(best_B_Absolute, X_test, c_test, sp_oracle)
    final_results.Huber_spoloss_test = spo_loss(best_B_Huber, X_test, c_test, sp_oracle)

    c_bar_test_preds = zeros(d_feasibleregion, n_test)
    for i = 1:n_test
        c_bar_test_preds[:, i] = c_bar_train
    end
    final_results.Baseline_spoloss_test = spo_loss(Matrix(1.0I, d_feasibleregion, d_feasibleregion), c_bar_test_preds, c_test, sp_oracle)


    # Now Hamming
    final_results.SPOplus_hammingloss_test = spo_loss(best_B_SPOplus, X_test, c_ham_test, sp_oracle)
    final_results.LS_hammingloss_test = spo_loss(best_B_leastSquares, X_test, c_ham_test, sp_oracle)
    final_results.SSVM_hammingloss_test = spo_loss(best_B_SSVM, X_test, c_ham_test, sp_oracle)

    if include_rf
        final_results.RF_hammingloss_test = spo_loss(Matrix(1.0I, d_feasibleregion, d_feasibleregion), rf_preds_test, c_ham_test, sp_oracle)
    else
        final_results.RF_hammingloss_test = missing
    end

    final_results.Absolute_hammingloss_test = spo_loss(best_B_Absolute, X_test, c_ham_test, sp_oracle)
    final_results.Huber_hammingloss_test = spo_loss(best_B_Huber, X_test, c_ham_test, sp_oracle)
    final_results.Baseline_hammingloss_test = spo_loss(Matrix(1.0I, d_feasibleregion, d_feasibleregion), c_bar_test_preds, c_ham_test, sp_oracle)

    final_results.zstar_avg_test = mean(z_test)

    return final_results
end